### 1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и 
реализовать функцию, записывающую собранные вакансии в созданную БД


In [1]:
from pymongo import MongoClient
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests

In [2]:
import re
import pandas as pd
import transliterate
import json

In [185]:
client  = MongoClient('localhost',27017)
db = client['vakans']
vakans_hh = db.vakans.vakans_hh
vakans_sj = db.vakans.vakans_sj
vakans = db.vakans

In [4]:
def page_table(link, header):
    result = pd.DataFrame(columns=['names', 'salarys_1', 'salarys_2', 'val', 'links', 'from'])

    global_name = 'https://hh.ru'
    main_link_last = f'{global_name}{link}'
    status = requests.get(main_link_last, headers = header).status_code
    if status != 200:
        return(result)
    response_last = requests.get(main_link_last, headers = header).text
    html_last = bs(response_last,'lxml')

    names = html_last.findAll('div',{'class':'resume-search-item__name'})
    names_list = []
    for i in names:
        names_list.append(i.getText())

    salarys = html_last.findAll('div',{'class':'vacancy-serp-item__row vacancy-serp-item__row_header'})
    salarys_all = []
    for i in salarys:
        salarys_2 = i.find('div',{'class':'vacancy-serp-item__compensation'})
        salarys_all.append(salarys_2)
    for i in range(len(salarys_all)):
        if salarys_all[i]:
            salarys_all[i] = salarys_all[i].getText()
    salarys_1 = []
    salarys_2 = []
    val = []
    for i in range(len(salarys_all)):
        salarys_1.append(0)
        salarys_2.append(0)
        val.append(None)
        if salarys_all[i]:
            hm = re.findall(r'^от_*', salarys_all[i])
            if hm:
                hm_num = re.sub(r'\xa0', '', salarys_all[i])
                hm_num = re.sub(r'от ', '', hm_num)
                hm_val = re.search(r'[\D]+', hm_num)
                hm_num = re.sub(r'[\D]+', '', hm_num)
                salarys_1[i] = int(hm_num)
                val[i] = hm_val[0]
            hm2 = re.findall(r'^до_*', salarys_all[i])
            if hm2:
                hm_num = re.sub(r'\xa0', '', salarys_all[i])
                hm_num = re.sub(r'до ', '', hm_num)
                hm_val = re.search(r'[\D]+', hm_num)
                hm_num = re.sub(r'[\D]+', '', hm_num)
                salarys_2[i] = int(hm_num)
                val[i] = hm_val[0]
            hm3 = re.search(r'-', salarys_all[i])
            if hm3:
                hm_num = re.split(r'-', salarys_all[i])
                hm_num1 = re.sub(r'\xa0', '', hm_num[0])
                hm_num2 = re.sub(r'\xa0', '', hm_num[1])
                hm_val = re.search(r'[\D]+', hm_num2)
                hm_num2 = re.sub(r'[\D]+', '', hm_num2)
                salarys_1[i] = int(hm_num1)
                salarys_2[i] = int(hm_num2)
                val[i] = hm_val[0]

    links_page = []
    for i in salarys:
        links_page1 = i.find('a',{'class':'bloko-link HH-LinkModifier'})
        links_page2 = links_page1['href']
        links_page.append(links_page2)
    
    result['names'] = names_list
    result['salarys_1'] = salarys_1
    result['salarys_2'] = salarys_2
    result['val'] = val
    result['links'] = links_page
    result['from'] = global_name
    
    return result

In [5]:
def page_table_two(link, header):
    result = pd.DataFrame(columns=['names', 'salarys_1', 'salarys_2', 'val', 'links', 'from'])

    global_name = 'https://superjob.ru'
    main_link_last = f'{global_name}{link}'
    status = requests.get(main_link_last, headers = header).status_code
    if status != 200:
        return(result)
    response_last = requests.get(main_link_last, headers = header).text
    html_last = bs(response_last,'lxml')
    
    names = html_last.findAll('div',{'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
    names_list = []
    for i in names:
        names_list.append(i.getText())

    salarys_all = html_last.findAll('span',{'class':'_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})
    for i in range(len(salarys_all)):
        salarys_all[i] = salarys_all[i].getText()   
    
    salarys_1 = []
    salarys_2 = []
    val = []
    for i in range(len(salarys_all)):
        hm = re.findall(r'^от_*', salarys_all[i])
        hm2 = re.findall(r'^до_*', salarys_all[i])
        hm3 = re.search(r'—', salarys_all[i])
        hm4 = re.search(r'По договорённости', salarys_all[i])
        if hm:
            hm_num = re.sub(r'\xa0', '', salarys_all[i])
            hm_num = re.sub(r'от', '', hm_num)
            hm_val = re.search(r'[\D]+', hm_num)
            hm_num = re.sub(r'[\D]+', '', hm_num)
            salarys_1.append(int(hm_num))
            salarys_2.append(0)
            val.append(hm_val[0])
        elif hm2:
            hm_num = re.sub(r'\xa0', '', salarys_all[i])
            hm_num = re.sub(r'до', '', hm_num)
            hm_val = re.search(r'[\D]+', hm_num)
            hm_num = re.sub(r'[\D]+', '', hm_num)
            salarys_1.append(0)
            salarys_2.append(int(hm_num))
            val.append(hm_val[0])
        elif hm3:
            hm_num = re.split(r'—', salarys_all[i])
            hm_num1 = re.sub(r'\xa0', '', hm_num[0])
            hm_num2 = re.sub(r'\xa0', '', hm_num[1])
            hm_val = re.search(r'[\D]+', hm_num2)
            hm_num2 = re.sub(r'[\D]+', '', hm_num2)
            salarys_1.append(int(hm_num1))
            salarys_2.append(int(hm_num2))
            val.append(hm_val[0])
        elif hm4:
            salarys_1.append(0)
            salarys_2.append(0)
            val.append(None)
        else:
            hm_num = re.sub(r'\xa0', '', salarys_all[i])
            hm_val = re.search(r'[\D]+', hm_num)
            hm_num = re.sub(r'[\D]+', '', hm_num)
            salarys_1.append(int(hm_num))
            salarys_2.append(int(hm_num))
            val.append(hm_val[0])
                
    links_page = []
    links_page1 = html_last.findAll('a',{'class':'_1QIBo'})
    for i in links_page1:
        links_page2 = i['href']
        links_page.append(f'{global_name}{links_page2}')
    
    result['names'] = names_list
    result['salarys_1'] = salarys_1
    result['salarys_2'] = salarys_2
    result['val'] = val
    result['links'] = links_page
    result['from'] = global_name
    
    return result

In [6]:
search_s = input('Введите название вакансии: ')

Введите название вакансии: Программист


In [7]:
def search_hh(pro):
    search = re.sub(r'\s', '+', pro)
    main_link = f'https://hh.ru/search/vacancy?area=1&clusters=true&enable_snippets=true&order_by=publication_time&text={search}&salary='

    header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
    status = requests.get(main_link, headers = header).status_code
    if status != 200:
        return('проверить соединение')
    response = requests.get(main_link, headers = header).text
    html = bs(response,'lxml')

    pager = html.findAll('span',{'class':'pager-item-not-in-short-range'})
    last_page = pager[-1].find('a',{'class':'bloko-button HH-Pager-Control', 'data-qa': 'pager-page'})
    link_last_num = last_page.getText()

    page_other = pd.DataFrame(columns=['names', 'salarys_1', 'salarys_2', 'val', 'links', 'from'])
    for i in range(int(link_last_num)):
        link_last = f'/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&order_by=publication_time&search_field=name&text={search}&page={i}'
        result = page_table(link_last, header)
        page_other = pd.merge(page_other, result, how='outer')
    return page_other

In [8]:
def search_sj(pro):
    search_two = transliterate.translit(pro, reversed=True)
    search_two = re.sub(r'\s', '-', search_two)
    main_link_two = f'https://superjob.ru/vakansii/{search_two}.html?geo%5Bt%5D%5B0%5D=4'

    header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
    status = requests.get(main_link_two, headers = header).status_code
    if status != 200:
        return('проверить соединение')
    response_two = requests.get(main_link_two, headers = header).text
    html_two = bs(response_two,'lxml')

    pager_two = html_two.findAll('div',{'class':'L1p51'})
    last_page_two = pager_two[0].findAll('span',{'class':'_3IDf-'})
    link_last_num_two = last_page_two[-2].getText()

    page_other_two = pd.DataFrame(columns=['names', 'salarys_1', 'salarys_2', 'val', 'links', 'from'])
    for i in range(1, int(link_last_num_two)+1):
        link_last = f'/vakansii/{search_two}.html?geo%5Bt%5D%5B0%5D=4&page={i}'
        result = page_table_two(link_last, header)
        page_other_two = pd.merge(page_other_two, result, how='outer')
    return page_other_two

In [223]:
page_other = search_hh(search_s)
page_other

,names,salarys_1,salarys_2,val,links,from
0,IOS Developer (Swift),250000,0,руб.,https://hh.ru/vacancy/35767587?query=%D0%9F%D1...,https://hh.ru
1,Удаленная работа Middle PHP программист (Symfo...,900,1300,USD,https://hh.ru/vacancy/35635063?query=%D0%9F%D1...,https://hh.ru
2,C# разработчик (Senior),200000,240000,руб.,https://hh.ru/vacancy/29832222?query=%D0%9F%D1...,https://hh.ru
3,Программист-разработчик (React \ ExtJS),0,0,None,https://hh.ru/vacancy/35763109?query=%D0%9F%D1...,https://hh.ru
4,Программист 1С,120000,0,руб.,https://hh.ru/vacancy/35762372?query=%D0%9F%D1...,https://hh.ru
...,...,...,...,...,...,...
1995,Консультант - программист 1С,0,0,None,https://hh.ru/vacancy/34972693?query=%D0%9F%D1...,https://hh.ru
1996,Программист 1С,120000,150000,руб.,https://hh.ru/vacancy/35710511?query=%D0%9F%D1...,https://hh.ru
1997,Технический переводчик/Программист Python,0,0,None,https://hh.ru/vacancy/35642592?query=%D0%9F%D1...,https://hh.ru
1998,Билд-инженер/Программист Python (Continuous In...,0,0,None,https://hh.ru/vacancy/35642567?query=%D0%9F%D1...,https://hh.ru


In [224]:
page_other_two = search_sj(search_s)
page_other_two

,names,salarys_1,salarys_2,val,links,from
0,PHP-Программист / Специалист по внедрению CRM ...,0,150000,₽,https://superjob.ru/vakansii/php-programmist-3...,https://superjob.ru
1,Web-программист / Веб-разработчик Битрикс Bitr...,0,220000,₽,https://superjob.ru/vakansii/web-programmist-1...,https://superjob.ru
2,Инженер-программист микроконтроллеров,0,0,None,https://superjob.ru/vakansii/inzhener-programm...,https://superjob.ru
3,Программист 1 С,0,0,None,https://superjob.ru/vakansii/programmist-1-s-2...,https://superjob.ru
4,Программист-консультант 1С,0,0,None,https://superjob.ru/vakansii/programmist-konsu...,https://superjob.ru
...,...,...,...,...,...,...
226,Веб-разработчик,60000,0,₽,https://superjob.ru/vakansii/veb-razrabotchik-...,https://superjob.ru
227,Data Scientist / Machine Learning Engineer,0,0,None,https://superjob.ru/vakansii/data-scientist-33...,https://superjob.ru
228,DevOps Engineer / Системный администратор,0,0,None,https://superjob.ru/vakansii/devops-engineer-3...,https://superjob.ru
229,Middle Golang Developer,0,0,None,https://superjob.ru/vakansii/middle-golang-dev...,https://superjob.ru


In [174]:
def load_mongo(page, name_mongo):
    j_page = page.to_json(orient='records')
    j_dict = json.loads(j_page)
    for i in j_dict:
        i['_id'] = i['links']
    name_mongo.insert_many(j_dict)

In [206]:
#load_mongo(page_other, vakans_hh)
#load_mongo(page_other_two, vakans_sj)

### 2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы


In [230]:
min_sal = int(input('Введите минимальный уровень зарплаты: '))

Введите минимальный уровень зарплаты: 100000


In [231]:
def min_s(name_collect, min_s):
    all_sal = name_collect.find({'salarys_1':{'$gt':min_s}}).sort('salarys_1')
    return all_sal

In [232]:
hh = min_s(vakans_hh, min_sal)
for i in hh:
    pprint(i)

{'_id': 'https://hh.ru/vacancy/35710088?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35710088?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Программист 1С',
 'salarys_1': 104000,
 'salarys_2': 120000,
 'val': ' руб.'}
{'_id': 'https://hh.ru/vacancy/35397269?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35397269?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Frontend-разработчик',
 'salarys_1': 110000,
 'salarys_2': 0,
 'val': ' руб.'}
{'_id': 'https://hh.ru/vacancy/35771420?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35771420?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Программист Битрикс24 CRM',
 'salary

 'val': ' руб.'}
{'_id': 'https://hh.ru/vacancy/35419892?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/35419892?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Программист 1С/Разработчик 1С',
 'salarys_1': 150000,
 'salarys_2': 0,
 'val': ' руб.'}
{'_id': 'https://hh.ru/vacancy/33865569?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/33865569?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Android разработчик',
 'salarys_1': 150000,
 'salarys_2': 200000,
 'val': ' руб.'}
{'_id': 'https://hh.ru/vacancy/33561038?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'from': 'https://hh.ru',
 'links': 'https://hh.ru/vacancy/33561038?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'names': 'Andro

In [233]:
s = min_s(vakans_sj, min_sal)
for i in s:
    pprint(i)

{'_id': 'https://superjob.ru/vakansii/programmist-33428710.html',
 'from': 'https://superjob.ru',
 'links': 'https://superjob.ru/vakansii/programmist-33428710.html',
 'names': 'Программист / Разработчик / ГИС специалист',
 'salarys_1': 110000,
 'salarys_2': 0,
 'val': '₽'}
{'_id': 'https://superjob.ru/vakansii/programmist-1s-33350624.html',
 'from': 'https://superjob.ru',
 'links': 'https://superjob.ru/vakansii/programmist-1s-33350624.html',
 'names': 'Программист 1С',
 'salarys_1': 120000,
 'salarys_2': 150000,
 'val': '₽'}
{'_id': 'https://superjob.ru/vakansii/programmist-pl-33369455.html',
 'from': 'https://superjob.ru',
 'links': 'https://superjob.ru/vakansii/programmist-pl-33369455.html',
 'names': 'Программист PL / SQL',
 'salarys_1': 120000,
 'salarys_2': 150000,
 'val': '₽'}
{'_id': 'https://superjob.ru/vakansii/programmist-razrabotchik-delphi-33369177.html',
 'from': 'https://superjob.ru',
 'links': 'https://superjob.ru/vakansii/programmist-razrabotchik-delphi-33369177.html',


### 3)*Написать функцию, 
которая будет добавлять в вашу базу данных только новые вакансии с сайта. Доработать функцию, которая будет обновлять старые вакансии.

In [199]:
def load_mongo_update(page, name_mongo):
    choose = input('Обновлять старые вакансии? y/n ')
    for i in range(len(page)):
        link = page.loc[i, 'links']
        all_sal = name_mongo.count_documents({'_id':{'$eq':link}})
        if choose == 'y':
            j_page = page.loc[i].to_json(orient='index')
            j_dict = json.loads(j_page)
            name_mongo.update_one({'links':link}, {'$set':j_dict})
            if all_sal is 0:
                print(link, 'new')
        elif (all_sal is 0) and (choose == 'n'):
            j_page = page.loc[i].to_json(orient='index')
            j_dict = json.loads(j_page)
            j_dict['_id'] = link
            name_mongo.insert_one(j_dict)
            print(link, 'new')

In [225]:
load_mongo_update(page_other, vakans_hh)
load_mongo_update(page_other_two, vakans_sj)

Обновлять старые вакансии? y/n n
https://hh.ru/vacancy/35767587?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82 new
Обновлять старые вакансии? y/n n
